In [19]:
!pip install pymorphy2

In [20]:
import pymorphy2
import nltk
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
def collect_text(url):
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
    script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  def is_date(o):
    x = o.split()
    if len(x) < 2:
      return False
    x = x[-2]
    if(len(x) == 4 and x[0] == '1' and x[1] == '9'):
      return True
    return False

  source = ''

  kill = False

  def TCHEKHOV(chunk):
    if 'ЧЕХОВ' in chunk:
      return True
    return False
  
  tch = ''

  for chunk in (chunks):
    if TCHEKHOV(chunk):
      tch = chunk
    if kill and chunk:
      source += chunk
      chunk = None
      break
    if is_date(chunk):
      source += (chunk)
      source += ' '
      chunk = None
      kill = True
    else:
      chunk = None


  erase = False

  def eraser(chunk):
    nonlocal erase
    if chunk == 'ПОИСК:':
      erase = True
    if erase:
      return False
    if not chunk:
      return False
    return True

  text = '\n'.join(chunk for chunk in chunks if eraser(chunk))

  sentences = nltk.tokenize.sent_tokenize(text)
  source = "Письмо " + tch + ' ' + source
  return source, sentences

In [66]:
def annotate_text(source, sentences):
  end_s = []
  for s in sentences:
    original = s
    original = original.lower()
    p = ',:@.-()/!'
    original_cleaned = original
    for i in p:
      original_cleaned = original_cleaned.replace(i, '')
    original_splitted = [ i for i in re.split(r'\W+',original) if i]
    
    lemmatized = []
    for j in original_splitted:
        lemmatized.append(morph.parse(j)[0].normal_form)
    tags = []
    for word in lemmatized:   
        tag = morph.parse(word)[0].tag
        tags.append(str(tag))
  end_s.append((source, s, original_splitted, lemmatized, tags))
  return (source, s, original_splitted, lemmatized, tags)

In [49]:
print(collect_text('http://apchekhov.ru/books/item/f00/s00/z0000020/st100.shtml'))

('Письмо 80*. О. Л. КНИППЕР - А. П. ЧЕХОВУ 23 января 1903 г. Москва', ['Голубчик мой, дорогой мой, второй день нет письма.', 'А последнее было такое нервное, нехорошее; что случилось?', 'Ты недоволен мной, сердишься на меня?', 'Я тебя раздражаю своими нелепыми письмами, сухими, рассеянными.', 'Я все это чувствую.', 'Прости, дорогой мой.', 'Я растрепалась душой.', 'Мне тяжело без тебя, нет опоры.', 'Я с тобой увереннее живу, крепче.', 'Ах, дусик мой... Если бы мне сейчас прижаться к тебе крепко, крепко, так, чтоб я слышала биение твоего сердца, чтоб ты мне говорил что-то ласковое, нежное и чтоб душа у меня смягчилась.', 'Когда мы увидимся, когда?!', 'А как вчера славно было!', 'Чистый, нетронутый, крупичатый снег, сосны, печальные, томящиеся, унылый их шум, золотистый закат, легкие облака, за Москвой-рекой какие-то татарские горы, точно предгорье Крыма, село, из-за холма любопытно торчит колокольня, на горизонте темный синеющий лес - так бы убежала на лыжах куда-то на простор, в ширь, т

In [74]:
corpora = []
for i in range(24, 354):
  if i == 193 or i == 194 or i == 202 or i == 203 or i == 204 or i == 205 or i == 313 or i == 326:
    continue
  print(i)
  if i < 100:
    url = 'http://apchekhov.ru/books/item/f00/s00/z0000020/st0' + str(i) + '.shtml'
  else:
    url = 'http://apchekhov.ru/books/item/f00/s00/z0000020/st' + str(i) + '.shtml'
  source, sentences = collect_text(url)
  annotated_text = annotate_text(source, sentences)
  corpora.append(annotated_text)

24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
195
196
197
198
199
200
201
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298


In [75]:
import json

data = corpora

with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)

with open('data.txt') as json_file:
    data = json.load(json_file)
    print(data)


[['Письмо 2*. О. Л. КНИППЕР-А. П. ЧЕХОВУ 27 ноября 1902 г. Москва', 'Твоя собака\nМамашу целую.', ['твоя', 'собака', 'мамашу', 'целую'], ['твой', 'собака', 'мамаша', 'целый'], ['ADJF,Apro inan,masc,sing,accs', 'NOUN,anim,femn sing,nomn', 'NOUN,anim,femn sing,nomn', 'ADJF,Qual inan,masc,sing,accs']], ['Письмо 3. А. П. ЧЕХОВ - О. Л. КНИППЕР 28 ноября 1902 г. Лозовая', 'Поклонись Маше (Поклонись Маше - Марии Павловне Чеховой)\nТвой А.', ['поклонись', 'маше', 'поклонись', 'маше', 'марии', 'павловне', 'чеховой', 'твой', 'а'], ['поклониться', 'маша', 'поклониться', 'маша', 'мария', 'павлович', 'чехов', 'твой', 'а'], ['INFN,perf,intr', 'NOUN,anim,femn,Name sing,nomn', 'INFN,perf,intr', 'NOUN,anim,femn,Name sing,nomn', 'NOUN,anim,femn,Name sing,nomn', 'NOUN,anim,masc,Patr sing,nomn', 'NOUN,anim,masc,Sgtm,Surn sing,nomn', 'ADJF,Apro inan,masc,sing,accs', 'CONJ']], ['Письмо 4*. О. Л. КНИППЕР - А. П. ЧЕХОВУ 28 ноября 1902 г. Москва', '...', [], [], []], ['Письмо 5 *. О. Л. КНИППЕР - А. П. ЧЕХОВУ 